## SQL File Cleanup Python Program
This repository contains a Python program designed to clean up unwanted instructions from a SQL file. The program was initially created to facilitate the process of importing databases into a local environment, aiming for improved efficiency and less rudimentary methods.

Although the current version is specifically tailored for a Drupal CMS structure, it can be extended with additional methods to make it useful in various scenarios.

#### How to use
To utilize the program, follow these steps:

1. Instantiate an object with the location of the SQL file. This will create a temporary copy of the file, which will be used for modifications, while the original file remains intact.
    
2. The program currently offers three main methods:
    - **removeSets:** This method receives a list of set instructions to be removed.
    - **changeToUTF8:** This method allows you to specify the collation to be replaced with UTF8 (default).
    - **removeDumbData:** This method receives a list of tables to be excluded. It is particularly useful for removing dumping data related to files that you do not wish to import into your local environment.

#### Notes
As a Python newbie, creating this program was an enjoyable learning experience. I would greatly appreciate any feedback or suggestions for improvement. Feel free to reach out and help me discover better ways to accomplish the desired functionality.


In [ ]:
import re
from datetime import datetime

class CleanSQLDB:
    def __init__(self, file):
        curr_dt = datetime.now()
        timestamp = int(round(curr_dt.timestamp()))
        self.file = file
        self.temp_file = file[:-4] + f"-{timestamp}.sql"
        with open(self.file, 'r') as file_r:
            with open(self.temp_file, 'w') as temp_file:
                temp_file.write(file_r.read())

    def removeSets(self, exclude_sets):
        filtered_lines = []
        with open(self.temp_file, 'r') as temp_file:
            filtered_lines = [line for line in temp_file if not any(excluded_item in line for excluded_item in exclude_sets)]
            with open(self.temp_file, 'w') as temp_file:
                for item in filtered_lines:
                    temp_file.write(item)

    def changeToUTF8(self, wrongUtf, wrongCollation):
        with open(self.temp_file, 'r') as temp_file:
            sql_contents = temp_file.read()
            sql_contents = re.sub(wrongUtf, 'utf8', sql_contents)
            sql_contents = re.sub(wrongCollation, 'utf8_general_ci', sql_contents)
            with open(self.temp_file, 'w') as temp_file_path:
                temp_file_path.write(sql_contents)
    
    def removeDumbData(self, excludedTables):
        with open(self.temp_file, 'r') as temp_file:
            sql_contents = temp_file.read()
            for excluded in excludedTables:    
                pattern = rf'LOCK TABLES `{excluded}` WRITE;.*?UNLOCK TABLES;'
                sql_contents = re.sub(pattern, '', sql_contents, flags=re.DOTALL)
            
            with open(self.temp_file, 'w') as temp_file_path:
                temp_file_path.write(sql_contents)


# Create an instance of the class
file = "/direction/of/file.sql"
obj = CleanSQLDB(file)

obj.removeSets(['SET @@SESSION.SQL_LOG_BIN', 'SET @MYSQLDUMP_TEMP_LOG_BIN','SET @@GLOBAL.GTID_PURGED'])
obj.changeToUTF8('utf8mb4','utf8_0900_ai_ci')
obj.removeDumbData([
    'node__field_etiqueta',
    'node_revision_field_etiqueta',
    'node__field_factura',
    'node_revision_field_factura',
    'node__field_comprobante_de_pago',
    'node_revision_field_comprobante_de_pago',
    'node__field_comprobante_de_encomienda',
    'node_revision_field_comprobante_de_encomienda',
    'watchdog'
])